# Programmatic implementation of AMDock pipeline

Testing the programmatic htvs via the `AMDock` workflow. `AMDock` is a nive GUI for autodock that works very well. Below is a test run of my `htvs_amdock` module that tries to programmatically replicate the same workflow as `AMDock`.

## Preparation

1. Download the test complex [1HSG protein with mk1 ligand](https://www.rcsb.org/structure/1HSG). Separate the protein and ligand pdb data in `pymol`.
2. Do not download the ligand structure from any other source. Use the one in the complex, as it is experimentally verified.
3. To reduce the possibility of false positives, the ligand PDB, once extracted from the experimentally verified crystal pose, was manually moved/rotated about in pymol by a random distance and angle.

In [1]:
import importmonkey
import os

# set OMP_NUM_THREADS to number of available CPUs. Best for docking a single complex quickly
cpus = os.cpu_count() or 1
os.environ["OMP_NUM_THREADS"] = str(cpus)
home = os.environ["HOME"]

importmonkey.add_path(os.path.join(home, "gitrepos/gromacs_sims/scripts"))
from htvs_amdock import run_htvs

PROTEIN_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1hsg_mk1/experiment/1hsg_protein.pdb")
LIGAND_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1hsg_mk1/experiment/mk1.pdb")

testjob = [{
            "protein_name": "1hsg_protein",
            "ligand_name": "mk1",
            "protein_pdb": PROTEIN_PDB,
            "ligand_pdb": LIGAND_PDB,
            "working_dir": os.path.join(home, "gitrepos/gromacs_sims/htvs/1hsg_mk1/job_1hsg_mk1_script_20251206"),
        }]

# Uncomment this to rerun the docking
#run_htvs(testjob)

## AutoLigand error (resolved)

======== AutoLigand Site Search (Automatic) FAILED STDOUT ========

======== AutoLigand Site Search (Automatic) FAILED STDERR ========
```python
Traceback (most recent call last):
  File "/usr/local/miniforge3/envs/htvs/bin/AutoLigand", line 7, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/miniforge3/envs/htvs/lib/python3.12/site-packages/AutoDockTools/AutoLigand.py", line 198, in main
    word = string.split(linee)
           ^^^^^^^^^^^^
AttributeError: module 'string' has no attribute 'split'
```

The `split()` function was deprecated and the functionality moved to a `xxx.split()` method during upgrade from python 2 to python 3. The `AutoLigand` script taken from [AutoDockTools_py3](https://github.com/Valdes-Tresanco-MS/AutoDockTools_py3) is supposed to translate the old `AutoLigand.py` written in python-2 to python-3, but fumbled it here. 


## Resolution
Raised this issue at the github repo, forked a copy of mine, then modded accordingly and it now works.

## Post-Processing

Tabulate the docking scores.

In [2]:
import os
home = os.environ["HOME"]
results_dir = os.path.join(home, "gitrepos/gromacs_sims/htvs/1hsg_mk1/job_1hsg_mk1_script_20251206")
# Scan all log files in the results_dir and print a table of ligand names and their best binding affinities. Use tabulate module for formatting.
from tabulate import tabulate
import re
results = []
for filename in os.listdir(results_dir):
    if not filename.endswith(".log"):
        continue
    log_path = os.path.join(results_dir, filename)
    # extract pose id from filename (try common patterns, fallback to filename stem)
    m = re.search(r'pose[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'site[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'_(\d+)\.log$', filename)
    pose_id = m.group(1) if m else os.path.splitext(filename)[0]
    with open(log_path, "r") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if line.startswith("-----+"):
                if i + 1 < len(lines):
                    parts = lines[i+1].split()
                    if len(parts) >= 2:
                        try:
                            affinity = float(parts[1])
                        except ValueError:
                            continue
                        results.append((pose_id, affinity))

# sort by affinity in increasing numeric order and format for display
results.sort(key=lambda x: x[1])
table = [(pid, f"{aff:.3f}") for pid, aff in results]
print(tabulate(table, headers=["Pose ID", "Best Binding Affinity (kcal/mol)"]))

  Pose ID    Best Binding Affinity (kcal/mol)
---------  ----------------------------------
        8                             -10.62
        5                             -10.57
        2                             -10.57
        9                             -10.56
       10                              -6.886
        1                              -6.855
        6                              -6.817
        7                              -6.764
        3                              -6.752
        4                              -5.879


## Visualize

Use `nglview` to checkout the best docked pose.

### TODO
Directly use `mdanalysis` to compare best docked ligand pose with the experimental one. Currently doing it with `pymol`, but it is better to just compute RMSD on jupyter and visualize both poses with nglview. 

In [3]:
ligand_id = "mk1"
best_pose_id, best_affinity = results[0]
best_pose_ligand_pdb = os.path.join(results_dir, f"{ligand_id}_pose_site{best_pose_id}.pdb")
protein_pdb = testjob[0]["protein_pdb"]
#use nglview to visualize the best pose
import nglview as nv
view = nv.NGLWidget()
view.add_component(protein_pdb, ext="pdb", default_representation="cartoon", name="Protein")
view.add_component(best_pose_ligand_pdb, ext="pdb", default_representation="ball+stick", name="Best Ligand Pose")
view.center()
view.display()
view

NGLWidget()

## Further Testing

Below is a concise report and a ready test set of **10 experimentally determined protein–ligand complexes (all proteins ≤200 residues)** you can use to validate docking workflows; the list and recommendations are drawn from public structural resources (RCSB PDB and curated datasets).

---

### Protein–ligand test set

| **PDB ID** | **Protein (residues)** | **Ligand (type)** | **Method / Resolution** | **Link** |
|---|---:|---|---|---|
| **1HEW** | **Hen egg‑white lysozyme (129)** | **Tri‑N‑acetylchitotriose (saccharide)** | X‑ray 1.75 Å | https://www.rcsb.org/structure/1HEW |
| **1FKB** | **FKBP12 (107)** | **Rapamycin (small molecule)** | X‑ray 1.70 Å | https://www.rcsb.org/structure/1FKB |
| **1HVR** | **HIV‑1 protease (99 per chain)** | **Nonpeptide cyclic urea inhibitor (small molecule)** | X‑ray 1.80 Å | https://www.rcsb.org/structure/1HVR |
| **1STP** | **Streptavidin monomer (159)** | **Biotin (small molecule)** | X‑ray 2.60 Å | https://www.rcsb.org/structure/1STP |
| **1AFK** | **Ribonuclease A (124)** | **Nucleotide analog (small molecule)** | X‑ray 1.70 Å | https://www.rcsb.org/structure/1AFK |
| **1TU2** | **Apocytochrome F and Plastocyanin Cu Metalloprotein (359)** | **HEC (HEME C ligand molecule)** | NMR  | https://www.rcsb.org/structure/1TU2 |
| **2UX7** | **Pseudoazurin + Amicyanon loop (122) Mg Metalloprotein @ pH 7.5** | **GOL ( Glycerol ligand molecule)** | X-ray 1.30 Å | https://www.rcsb.org/structure/2UX7 |
| **1MH1** | **Small G-Protein (183) Cu Metalloprotein** | **GNP ( PHOSPHOAMINOPHOSPHONIC ACID-GUANYLATE ligand molecule)** | X-ray 1.38 Å | https://www.rcsb.org/structure/1MH1 |
| **2K0F** | **Calmodulin (148)** | **Calmodulin‑binding peptide (peptide ligand)** | NMR (solution) | https://www.rcsb.org/structure/2K0F |
| **1YCR** | **MDM2 (109)** | **p53 transactivation peptide (peptide ligand)** | X‑ray 2.60 Å | https://www.rcsb.org/structure/1YCR |
| **1BRS** | **Barnase (110) / Barstar (89)** | **Protein inhibitor (protein–protein complex)** | X‑ray 2.00 Å | https://www.rcsb.org/structure/1BRS |
| **1STN** | **Staphylococcal nuclease (149)** | **Small‑molecule binding site available** | X‑ray 1.70 Å | https://www.rcsb.org/structure/1STN |
| **1DFJ** | **Ribonuclease A (124)** | **Bound inhibitor (small/peptide‑like)** | X‑ray 2.50 Å | https://www.rcsb.org/structure/1DFJ |

### Notes and reasoning

- **Selection criteria:** I prioritized **experimentally validated** entries from public structural repositories and curated datasets, and constrained targets to **≤200 residues** to keep receptors small and tractable for docking and sampling. The RCSB PDB is the primary source for coordinates, experimental method, and resolution. PLBD and curated collections (e.g., PL‑REX) provide complementary lists and affinity/validation metadata useful for benchmarking.
- **Ligand diversity:** The set intentionally spans **small molecules (biotin, rapamycin, inhibitors), nucleotides, saccharides, peptides, and a protein inhibitor** to exercise different docking challenges: rigid small‑molecule fits, flexible sugar recognition, peptide backbone sampling, and protein–protein interfaces.
- **Why these are useful:** **Small proteins reduce conformational search space**, making it easier to isolate docking algorithm performance. Peptide and protein‑protein cases test sampling and scoring for larger, flexible interfaces; small‑molecule cases test pose prediction and scoring accuracy.
- **Practical preparation steps:** Download coordinate files (PDBx/mmCIF) and validation reports from RCSB; extract **chain IDs**, **ligand Chemical Component IDs**, and experimental metadata before preprocessing. Standardize protonation, remove or retain crystallographic waters per your protocol, and handle alternate conformers consistently.
- **Validation metrics:** For **small molecules** compute heavy‑atom RMSD to deposited ligand coordinates; for **peptides/protein interfaces** use backbone RMSD and interface contact recovery (e.g., fraction of native contacts). Where available, use experimental affinity data from PLBD or curated datasets to correlate docking scores with binding strength.
- **Next steps I can provide:** I can (a) expand the set with additional small‑molecule complexes, (b) extract **ligand Chemical Component IDs and recommended receptor chain selections** for each PDB entry, or (c) produce a CSV of the table for local use.

**Key takeaway:** This mixed set gives **broad coverage of docking scenarios** while keeping receptors small (≤200 residues), enabling focused benchmarking of sampling and scoring components using experimentally validated structures.

## Complex 1HEW_NAG


In [1]:
import importmonkey
import os
home = os.environ["HOME"]

importmonkey.add_path(os.path.join(home, "gitrepos/gromacs_sims/scripts"))
from htvs_amdock import run_htvs

PROTEIN_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1hew_nag/experiment/1HEW_expt.pdb")
LIGAND_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1hew_nag/experiment/nag.pdb")

testjob = [{
            "protein_name": "1hew_protein",
            "ligand_name": "nag",
            "protein_pdb": PROTEIN_PDB,
            "ligand_pdb": LIGAND_PDB,
            "working_dir": os.path.join(home, "gitrepos/gromacs_sims/htvs/1hew_nag/job_1hew_nag_script_20251208"),
        }]

# Uncomment this to rerun the docking
#run_htvs(testjob, verbose=True)

In [2]:
import os
home = os.environ["HOME"]
results_dir = os.path.join(home, "gitrepos/gromacs_sims/htvs/1hew_nag/job_1hew_nag_script_20251208")
# Scan all log files in the results_dir and print a table of ligand names and their best binding affinities. Use tabulate module for formatting.
from tabulate import tabulate
import re
results = []
for filename in os.listdir(results_dir):
    if not filename.endswith(".log"):
        continue
    log_path = os.path.join(results_dir, filename)
    # extract pose id from filename (try common patterns, fallback to filename stem)
    m = re.search(r'pose[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'site[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'_(\d+)\.log$', filename)
    pose_id = m.group(1) if m else os.path.splitext(filename)[0]
    with open(log_path, "r") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if line.startswith("-----+"):
                if i + 1 < len(lines):
                    parts = lines[i+1].split()
                    if len(parts) >= 2:
                        try:
                            affinity = float(parts[1])
                        except ValueError:
                            continue
                        results.append((pose_id, affinity))

# sort by affinity in increasing numeric order and format for display
results.sort(key=lambda x: x[1])
table = [(pid, f"{aff:.3f}") for pid, aff in results]
print(tabulate(table, headers=["Pose ID", "Best Binding Affinity (kcal/mol)"]))

  Pose ID    Best Binding Affinity (kcal/mol)
---------  ----------------------------------
        7                              -7.044
        3                              -7.016
        2                              -6.982
        1                              -6.973
        5                              -6.964
        6                              -6.886
        4                              -6.861
        8                              -5.453


In [11]:
ligand_id = "nag"
best_pose_id, best_affinity = results[0]
ligand_expt_pose = os.path.join(home, "gitrepos/gromacs_sims/htvs/1hew_nag/experiment/nag_expt.pdb")
best_pose_ligand_pdb = os.path.join(results_dir, f"{ligand_id}_pose_site{best_pose_id}.pdb")
protein_pdb = testjob[0]["protein_pdb"]
#use nglview to visualize the best pose
import nglview as nv
from IPython.display import display, Image
view = nv.NGLWidget()

view.add_component(protein_pdb, ext="pdb", default_representation="cartoon", name="Protein")
view.add_component(ligand_expt_pose, ext="pdb", default_representation="hyperball", name="Experimental Ligand")
view.add_component(best_pose_ligand_pdb, ext="pdb", default_representation="hyperball", name="Best Ligand Pose")
view.center()
view

NGLWidget()

## Multiple complexes in Parallel

This is best done in script. The environment variables are properly exported that way.

We will now try to dock three complexes concurrently in parallel.

1. 1fkb_rap
2. 1stp_btn
3. 1afk_pap

In [1]:
#/usr/bin/env python
import importmonkey
import os
home = os.environ["HOME"]

importmonkey.add_path(os.path.join(home, "gitrepos/gromacs_sims/scripts"))
from htvs_amdock import run_htvs

PROTEIN_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1fkb_rap/1FKB_expt.pdb")
LIGAND_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1fkb_rap/rap.pdb")

docking_jobs = [{
            "protein_name": "1fkb_protein",
            "ligand_name": "rap",
            "protein_pdb": PROTEIN_PDB,
            "ligand_pdb": LIGAND_PDB,
            "working_dir": os.path.join(home, "gitrepos/gromacs_sims/htvs/1fkb_rap/job_1fkb_rap_script_20251208"),
        }]

PROTEIN_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1stp_btn/1STP_expt.pdb")
LIGAND_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1stp_btn/btn.pdb")

docking_jobs.append({
            "protein_name": "1stp_protein",
            "ligand_name": "btn",
            "protein_pdb": PROTEIN_PDB,
            "ligand_pdb": LIGAND_PDB,
            "working_dir": os.path.join(home, "gitrepos/gromacs_sims/htvs/1stp_btn/job_1stp_btn_script_20251208"),
        })


PROTEIN_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1afk_pap/1afk_expt.pdb")
LIGAND_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1afk_pap/pap.pdb")

docking_jobs.append({
            "protein_name": "1afk_protein",
            "ligand_name": "pap",
            "protein_pdb": PROTEIN_PDB,
            "ligand_pdb": LIGAND_PDB,
            "working_dir": os.path.join(home, "gitrepos/gromacs_sims/htvs/1afk_pap/job_1afk_pap_script_20251208"),
        })

# The docking ran in script


### 1fkb_rap

In [4]:
import os
home = os.environ["HOME"]
results_dir = os.path.join(home, "gitrepos/gromacs_sims/htvs/1fkb_rap/job_1fkb_rap_script_20251208")
# Scan all log files in the results_dir and print a table of ligand names and their best binding affinities. Use tabulate module for formatting.
from tabulate import tabulate
import re
results = []
for filename in os.listdir(results_dir):
    if not filename.endswith(".log"):
        continue
    log_path = os.path.join(results_dir, filename)
    # extract pose id from filename (try common patterns, fallback to filename stem)
    m = re.search(r'pose[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'site[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'_(\d+)\.log$', filename)
    pose_id = m.group(1) if m else os.path.splitext(filename)[0]
    with open(log_path, "r") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if line.startswith("-----+"):
                if i + 1 < len(lines):
                    parts = lines[i+1].split()
                    if len(parts) >= 2:
                        try:
                            affinity = float(parts[1])
                        except ValueError:
                            continue
                        results.append((pose_id, affinity))

# sort by affinity in increasing numeric order and format for display
results.sort(key=lambda x: x[1])
table = [(pid, f"{aff:.3f}") for pid, aff in results]
print(tabulate(table, headers=["Pose ID", "Best Binding Affinity (kcal/mol)"]))

  Pose ID    Best Binding Affinity (kcal/mol)
---------  ----------------------------------
        1                             -10.88
        3                             -10.85
        8                             -10.85
        5                             -10.84
        2                             -10.82
        7                             -10.82
        6                              -8.043
        9                              -6.723
        4                              -6.196
       10                              -5.753


In [16]:
ligand_id = "rap"
best_pose_id, best_affinity = results[0]
ligand_expt_pose = os.path.join(home, "gitrepos/gromacs_sims/htvs/1fkb_rap/experiment/rap_expt.pdb")
best_pose_ligand_pdb = os.path.join(results_dir, f"{ligand_id}_pose_site{best_pose_id}.pdb")
protein_pdb = docking_jobs[0]["protein_pdb"]
#use nglview to visualize the best pose
import nglview as nv
from IPython.display import display, Image
view = nv.NGLWidget()

view.add_component(protein_pdb, ext="pdb", default_representation="cartoon", name="Protein")
view.add_component(ligand_expt_pose, ext="pdb", default_representation="licorice", name="Experimental Ligand")
view.add_component(best_pose_ligand_pdb, ext="pdb", default_representation="hyperball", name="Best Ligand Pose")
view.center()
view

NGLWidget()

The match is nearly perfect here. Excellent docking score implies good experimental match.

### 1stp_btn

In [18]:
import os
home = os.environ["HOME"]
results_dir = os.path.join(home, "gitrepos/gromacs_sims/htvs/1stp_btn/job_1stp_btn_script_20251208")
# Scan all log files in the results_dir and print a table of ligand names and their best binding affinities. Use tabulate module for formatting.
from tabulate import tabulate
import re
results = []
for filename in os.listdir(results_dir):
    if not filename.endswith(".log"):
        continue
    log_path = os.path.join(results_dir, filename)
    # extract pose id from filename (try common patterns, fallback to filename stem)
    m = re.search(r'pose[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'site[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'_(\d+)\.log$', filename)
    pose_id = m.group(1) if m else os.path.splitext(filename)[0]
    with open(log_path, "r") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if line.startswith("-----+"):
                if i + 1 < len(lines):
                    parts = lines[i+1].split()
                    if len(parts) >= 2:
                        try:
                            affinity = float(parts[1])
                        except ValueError:
                            continue
                        results.append((pose_id, affinity))

# sort by affinity in increasing numeric order and format for display
results.sort(key=lambda x: x[1])
table = [(pid, f"{aff:.3f}") for pid, aff in results]
print(tabulate(table, headers=["Pose ID", "Best Binding Affinity (kcal/mol)"]))

  Pose ID    Best Binding Affinity (kcal/mol)
---------  ----------------------------------
        5                              -4.242
        9                              -4.189
        6                              -4.14
        4                              -4.071
        7                              -3.961
        2                              -3.861
        3                              -3.77
       10                              -3.756
        8                              -3.712
        1                              -3.422


In [19]:
ligand_id = "btn"
best_pose_id, best_affinity = results[0]
ligand_expt_pose = os.path.join(home, "gitrepos/gromacs_sims/htvs/1stp_btn/experiment/btn_expt.pdb")
best_pose_ligand_pdb = os.path.join(results_dir, f"{ligand_id}_pose_site{best_pose_id}.pdb")
protein_pdb = docking_jobs[1]["protein_pdb"]
#use nglview to visualize the best pose
import nglview as nv
from IPython.display import display, Image
view = nv.NGLWidget()

view.add_component(protein_pdb, ext="pdb", default_representation="cartoon", name="Protein")
view.add_component(ligand_expt_pose, ext="pdb", default_representation="licorice", name="Experimental Ligand")
view.add_component(best_pose_ligand_pdb, ext="pdb", default_representation="hyperball", name="Best Ligand Pose")
view.center()
view

NGLWidget()

Docking score is crap. The matching is poor, with the docked ligand lying outside the protein complex. Poor score means poor matching.

### 1afk_pap

In [20]:
import os
home = os.environ["HOME"]
results_dir = os.path.join(home, "gitrepos/gromacs_sims/htvs/1afk_pap/job_1afk_pap_script_20251208")
# Scan all log files in the results_dir and print a table of ligand names and their best binding affinities. Use tabulate module for formatting.
from tabulate import tabulate
import re
results = []
for filename in os.listdir(results_dir):
    if not filename.endswith(".log"):
        continue
    log_path = os.path.join(results_dir, filename)
    # extract pose id from filename (try common patterns, fallback to filename stem)
    m = re.search(r'pose[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'site[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'_(\d+)\.log$', filename)
    pose_id = m.group(1) if m else os.path.splitext(filename)[0]
    with open(log_path, "r") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if line.startswith("-----+"):
                if i + 1 < len(lines):
                    parts = lines[i+1].split()
                    if len(parts) >= 2:
                        try:
                            affinity = float(parts[1])
                        except ValueError:
                            continue
                        results.append((pose_id, affinity))

# sort by affinity in increasing numeric order and format for display
results.sort(key=lambda x: x[1])
table = [(pid, f"{aff:.3f}") for pid, aff in results]
print(tabulate(table, headers=["Pose ID", "Best Binding Affinity (kcal/mol)"]))

  Pose ID    Best Binding Affinity (kcal/mol)
---------  ----------------------------------
        1                              -5.974
        2                              -5.945
        3                              -5.935
        9                              -5.886
        8                              -5.482
        4                              -5.306
        5                              -5.283
        7                              -5.154
        6                              -5.102


In [21]:
ligand_id = "pap"
best_pose_id, best_affinity = results[0]
ligand_expt_pose = os.path.join(home, "gitrepos/gromacs_sims/htvs/1afk_pap/experiment/pap_expt.pdb")
best_pose_ligand_pdb = os.path.join(results_dir, f"{ligand_id}_pose_site{best_pose_id}.pdb")
protein_pdb = docking_jobs[2]["protein_pdb"]
#use nglview to visualize the best pose
import nglview as nv
from IPython.display import display, Image
view = nv.NGLWidget()

view.add_component(protein_pdb, ext="pdb", default_representation="cartoon", name="Protein")
view.add_component(ligand_expt_pose, ext="pdb", default_representation="licorice", name="Experimental Ligand")
view.add_component(best_pose_ligand_pdb, ext="pdb", default_representation="hyperball", name="Best Ligand Pose")
view.center()
view

NGLWidget()

Three nearly degenerate-scored docking poses, all poor scores, with poor matching with experiment, but better than the previous case.

**TODO** Check the two complexes with `AMDock` GUI to see if the results are consistent with the python workflow.

1. 1stb_btn
2. 1afk_pap

### Report

1stb_btn docked nearly perfectly in `AMDock`, suggesting that my python workflow needs improvement. Possible improvements are:
1. Hacking the default settings of `AMDock` workflow to match the python module better. **Update:** See below for observations.
2. Using more cores while running `vina`. **Update:** Doesn't help. See below.

The 1afk_pap complex ran about as poorly with `AMDock` as with the python module.

It may be that docking scores above $-7\; kcal/mol$ are just completely unreliable.

Reran 1stb_btn serially with all CPUs for `vina`. Got the following results:

In [3]:
import os
home = os.environ["HOME"]
results_dir = os.path.join(home, "gitrepos/gromacs_sims/htvs/1stp_btn_rerun/job_1stp_btn_script_20251208")
# Scan all log files in the results_dir and print a table of ligand names and their best binding affinities. Use tabulate module for formatting.
from tabulate import tabulate
import re
results = []
for filename in os.listdir(results_dir):
    if not filename.endswith(".log"):
        continue
    log_path = os.path.join(results_dir, filename)
    # extract pose id from filename (try common patterns, fallback to filename stem)
    m = re.search(r'pose[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'site[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'_(\d+)\.log$', filename)
    pose_id = m.group(1) if m else os.path.splitext(filename)[0]
    with open(log_path, "r") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if line.startswith("-----+"):
                if i + 1 < len(lines):
                    parts = lines[i+1].split()
                    if len(parts) >= 2:
                        try:
                            affinity = float(parts[1])
                        except ValueError:
                            continue
                        results.append((pose_id, affinity))

# sort by affinity in increasing numeric order and format for display
results.sort(key=lambda x: x[1])
table = [(pid, f"{aff:.3f}") for pid, aff in results]
print(tabulate(table, headers=["Pose ID", "Best Binding Affinity (kcal/mol)"]))

  Pose ID    Best Binding Affinity (kcal/mol)
---------  ----------------------------------
        6                              -4.15
        7                              -4.118
        8                              -4.036
        9                              -4.025
        5                              -3.98
        4                              -3.916
        3                              -3.853
       10                              -3.802
        1                              -3.789
        2                              -3.534


It's still crap! So using more CPUs for vina isn't the answer.

Next is to hack `AmDock` better. I compared the amdock log file with log files from the python workflow and saw that a key difference is in the `propka` options. `AMDock` adds extra options that are not added by default in the python workflow.

**Conclusion:** Review the `propka` section in the `AMDock` log files from the docking of `1stp_btn` and compare with `propka` logs from the docking of the same complex using the python workflow. Implement the extra options into the python workflow and rerun. As always, it's the protonation of the ligand where things seem to go sideways... 


## Re-test

I implemented the above suggestions on 20251210. Now rerunning `1stp_btn` complex and compare with `AMDock` results.

In [ ]:
import importmonkey
import os
home = os.environ["HOME"]
importmonkey.add_path(os.path.join(home, "gitrepos/gromacs_sims/scripts"))
from htvs_amdock import run_htvs

PROTEIN_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1stp_btn/1STP_expt.pdb")
LIGAND_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1stp_btn/btn.pdb")

new_1stp_btn = [{
            "protein_name": "1stp_protein",
            "ligand_name": "btn",
            "protein_pdb": PROTEIN_PDB,
            "ligand_pdb": LIGAND_PDB,
            "working_dir": os.path.join(home, "gitrepos/gromacs_sims/htvs/1stp_btn/job_1stp_btn_script_20251210"),
        }]

# Uncomment this to rerun the docking
run_htvs(new_1stp_btn, verbose=True)

In [2]:
import os
home = os.environ["HOME"]
results_dir = os.path.join(home, "gitrepos/gromacs_sims/htvs/1stp_btn/job_1stp_btn_script_20251210")
# Scan all log files in the results_dir and print a table of ligand names and their best binding affinities. Use tabulate module for formatting.
from tabulate import tabulate
import re
results = []
for filename in os.listdir(results_dir):
    if not filename.endswith(".log"):
        continue
    log_path = os.path.join(results_dir, filename)
    # extract pose id from filename (try common patterns, fallback to filename stem)
    m = re.search(r'pose[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'site[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'_(\d+)\.log$', filename)
    pose_id = m.group(1) if m else os.path.splitext(filename)[0]
    with open(log_path, "r") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if line.startswith("-----+"):
                if i + 1 < len(lines):
                    parts = lines[i+1].split()
                    if len(parts) >= 2:
                        try:
                            affinity = float(parts[1])
                        except ValueError:
                            continue
                        results.append((pose_id, affinity))

# sort by affinity in increasing numeric order and format for display
results.sort(key=lambda x: x[1])
table = [(pid, f"{aff:.3f}") for pid, aff in results]
print(tabulate(table, headers=["Pose ID", "Best Binding Affinity (kcal/mol)"]))

  Pose ID    Best Binding Affinity (kcal/mol)
---------  ----------------------------------
        2                              -6.936
        3                              -6.261
        7                              -6.243
        4                              -6.026
        1                              -5.844
        5                              -5.815
        8                              -5.288
       10                              -4.585
        6                              -4.522
        9                              -4.446


In [4]:
ligand_id = "btn"
best_pose_id, best_affinity = results[0]
ligand_expt_pose = os.path.join(home, "gitrepos/gromacs_sims/htvs/1stp_btn/btn_expt.pdb")
best_pose_ligand_pdb = os.path.join(results_dir, f"{ligand_id}_pose_site{best_pose_id}.pdb")
protein_pdb = new_1stp_btn[0]["protein_pdb"]
#use nglview to visualize the best pose
import nglview as nv
from IPython.display import display, Image
view = nv.NGLWidget()

view.add_component(protein_pdb, ext="pdb", default_representation="cartoon", name="Protein")
view.add_component(ligand_expt_pose, ext="pdb", default_representation="licorice", name="Experimental Ligand")
view.add_component(best_pose_ligand_pdb, ext="pdb", default_representation="hyperball", name="Best Ligand Pose")
view.center()
view    

NGLWidget()

The matching is much, much better for `1stp_btn` at least.

Now, rerunning for `1afk_pap` complex.

In [ ]:
import importmonkey
import os
home = os.environ["HOME"]
importmonkey.add_path(os.path.join(home, "gitrepos/gromacs_sims/scripts"))
from htvs_amdock import run_htvs

PROTEIN_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1afk_pap/1afk_expt.pdb")
LIGAND_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1afk_pap/pap.pdb")

new_1afk_pap = [{
            "protein_name": "1afk_protein",
            "ligand_name": "pap",
            "protein_pdb": PROTEIN_PDB,
            "ligand_pdb": LIGAND_PDB,
            "working_dir": os.path.join(home, "gitrepos/gromacs_sims/htvs/1afk_pap/job_1afk_pap_script_20251210"),
        }]

# Uncomment this to rerun the docking
run_htvs(new_1afk_pap, verbose=False)

In [6]:
import os
home = os.environ["HOME"]
results_dir = os.path.join(home, "gitrepos/gromacs_sims/htvs/1afk_pap/job_1afk_pap_script_20251210")
# Scan all log files in the results_dir and print a table of ligand names and their best binding affinities. Use tabulate module for formatting.
from tabulate import tabulate
import re
results = []
for filename in os.listdir(results_dir):
    if not filename.endswith(".log"):
        continue
    log_path = os.path.join(results_dir, filename)
    # extract pose id from filename (try common patterns, fallback to filename stem)
    m = re.search(r'pose[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'site[_-]?(\d+)', filename, re.IGNORECASE) or \
        re.search(r'_(\d+)\.log$', filename)
    pose_id = m.group(1) if m else os.path.splitext(filename)[0]
    with open(log_path, "r") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if line.startswith("-----+"):
                if i + 1 < len(lines):
                    parts = lines[i+1].split()
                    if len(parts) >= 2:
                        try:
                            affinity = float(parts[1])
                        except ValueError:
                            continue
                        results.append((pose_id, affinity))

# sort by affinity in increasing numeric order and format for display
results.sort(key=lambda x: x[1])
table = [(pid, f"{aff:.3f}") for pid, aff in results]
print(tabulate(table, headers=["Pose ID", "Best Binding Affinity (kcal/mol)"]))

  Pose ID    Best Binding Affinity (kcal/mol)
---------  ----------------------------------
        4                              -7.056
        7                              -6.821
        6                              -6.793
        3                              -6.755
        2                              -6.718
        5                              -6.487
        8                              -5.939
        1                              -5.938
        9                              -5.895
       10                              -5.456


In [7]:
ligand_id = "pap"
best_pose_id, best_affinity = results[0]
ligand_expt_pose = os.path.join(home, "gitrepos/gromacs_sims/htvs/1afk_pap/pap_expt.pdb")
ligand_amdock_pose = os.path.join(home, "gitrepos/gromacs_sims/htvs/1afk_pap/amdock/1afk_pap_amdock_20251208/results/pap_1afk_expt_out04.pdb")
best_pose_ligand_pdb = os.path.join(results_dir, f"{ligand_id}_pose_site{best_pose_id}.pdb")
protein_pdb = new_1afk_pap[0]["protein_pdb"]
#use nglview to visualize the best pose
import nglview as nv
from IPython.display import display, Image
view = nv.NGLWidget()

view.add_component(protein_pdb, ext="pdb", default_representation="cartoon", name="Protein")
view.add_component(ligand_expt_pose, ext="pdb", default_representation="licorice", name="Experimental Ligand")
view.add_component(best_pose_ligand_pdb, ext="pdb", default_representation="hyperball", name="Best Ligand Pose (AmDock)")
view.add_component(ligand_amdock_pose, ext="pdb", default_representation="hyperball", name="Best Ligand Pose (AmDock)")
view.center()
view

NGLWidget()

The `htvs_amdock` python pipeline agrees well with `AMDock`, but neither agree well with the experimental pose.


## Rerun 1fkb_rap
Another complex reran with output parsing.

In [1]:
#/usr/bin/env python
import importmonkey
import os
home = os.environ["HOME"]

importmonkey.add_path(os.path.join(home, "gitrepos/gromacs_sims/scripts"))
from htvs_amdock import run_htvs

PROTEIN_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1fkb_rap/1FKB_expt.pdb")
LIGAND_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1fkb_rap/rap.pdb")

new_1fkb_rap = [{
            "protein_name": "1fkb_protein",
            "ligand_name": "rap",
            "protein_pdb": PROTEIN_PDB,
            "ligand_pdb": LIGAND_PDB,
            "working_dir": os.path.join(home, "gitrepos/gromacs_sims/htvs/1fkb_rap/job_1fkb_rap_script_20251210"),
        }]


# The docking 
#run_htvs(new_1fkb_rap, verbose=False)

In [2]:
from htvs_amdock import analyze_htvs_results
analyze_htvs_results(new_1fkb_rap)


--- Summary for 1fkb_protein + rap ---

=== DOCKING LOG SUMMARY ===

Log file: /home/daneel/gitrepos/gromacs_sims/htvs/1fkb_rap/job_1fkb_rap_script_20251210/1fkb_protein_rap_pipeline.log

Detected tools / headers:
  - propka_header: INFO:PROPKA information:
  - pdb2pqr: 3.6.1
  - vina: present

PDB2PQR / structure info:
  Residues: 107, Atoms: 1021

PROPKA / pKa application:
  pH applied: 7.4
  Calculated pI (folded): 8.67
  ======== PDB2PQR/PROPKA STDOUT ========
  pKa entries parsed: 36
  Lowest pKa residues (up to 5):
         ASP 11A  pKa=1.99
         ASP 37A  pKa=2.13
         ASP 79A  pKa=2.71
        ASP 100A  pKa=2.79
         C- 107A  pKa=2.89
  Highest pKa residues (up to 5):
         ARG 71A  pKa=14.39
         ARG 42A  pKa=14.31
         ARG 57A  pKa=13.56
         ARG 40A  pKa=12.83
         ARG 13A  pKa=12.46

AutoLigand (Automatic) site search:
  Rank 1: Volume=241.0, EPV=-0.1949
  Rank 2: Volume=307.0, EPV=-0.1561
  Rank 3: Volume=275.0, EPV=-0.1426
  Rank 4: Volume=3

In [9]:
ligand_id = "rap"
best_pose_id = 1
ligand_expt_pose = os.path.join(home, "gitrepos/gromacs_sims/htvs/1fkb_rap/rap_expt.pdb")
best_pose_ligand_pdb = os.path.join(os.path.join(home, "gitrepos/gromacs_sims/htvs/1fkb_rap/job_1fkb_rap_script_20251210"), f"{ligand_id}_pose_site{best_pose_id}.pdb")
protein_pdb = new_1fkb_rap[0]["protein_pdb"]
#use nglview to visualize the best pose
import nglview as nv
from IPython.display import display, Image
view = nv.NGLWidget()

view.add_component(protein_pdb, ext="pdb", default_representation="cartoon", name="Protein")
view.add_component(ligand_expt_pose, ext="pdb", default_representation="licorice", name="Experimental Ligand")
view.add_component(best_pose_ligand_pdb, ext="pdb", default_representation="hyperball", name="Best Ligand Pose")
view.center()
view

NGLWidget()

## Report (20251210)

So far so good. `AMDock` and `htvs_amdock` are more-or-less in agreement for simple proteins complexed with ligands.

**TODO**
1. Proteins with metals (check `AMDock` options and implement modifications in `htvs_amdock`). Try the metalloprotein complexes from the table above.
   
   * Look at AMDock source code files `input_tab.py` and the `Fix_PQR` class in `tools.py`
   * Look at the code involving the `Keep Metals in Receptors:` option.
   * The idea is simple (in principle). `PDB2PQR/PROPKA` removes all metals from the input pdb while protonating. So the protonated PDB needs proper `TER`, followed by simply copy-pasting the metal entries from the original PDB. Only then can `prepare_receptor4` be run with `-U, -p XXX, -C` flags


2. `Autodock4Zn` with Zinc fingers
3. Protein-peptide complexes
4. Off target (protein-protein) docking
5. Possibly docking peptides/ligands with DNA segments (optional).

### 1MH1_GNP First metalloprotein-ligand docking



In [1]:
import importmonkey
import os

home = os.environ["HOME"]

importmonkey.add_path(os.path.join(home, "gitrepos/gromacs_sims/scripts"))
from htvs_amdock import run_htvs

PROTEIN_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1mh1_gnp/1MH1_protein.pdb")
LIGAND_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1mh1_gnp/gnp.pdb")

metalloprotein_job = [{
            "protein_name": "1mh1_protein",
            "ligand_name": "gnp",
            "protein_pdb": PROTEIN_PDB,
            "ligand_pdb": LIGAND_PDB,
            "working_dir": os.path.join(home, "gitrepos/gromacs_sims/htvs/1mh1_gnp/job_1mh1_gnp_script_20251212"),
            "keep_metals": True,
        }]

# Uncomment this to rerun the docking
run_htvs(metalloprotein_job, verbose=True)


--- Starting HTVS Parallel Execution ---
Vina internal cores per job: 12
Maximum simultaneous HTVS jobs: 1
Total jobs requested: 1
Running pdb2pqr: pdb2pqr --keep-chain --noopt --ff=AMBER --with-ph 7.4 /home/daneel/gitrepos/gromacs_sims/htvs/1mh1_gnp/1MH1_protein.pdb /home/daneel/gitrepos/gromacs_sims/htvs/1mh1_gnp/job_1mh1_gnp_script_20251212/1mh1_protein_h.pqr --nodebump -k --protonate-all --titration-state-method=propka --pH 7.4 -f /home/daneel/gitrepos/gromacs_sims/htvs/1mh1_gnp/job_1mh1_gnp_script_20251212/1mh1_protein_h_h.pdb
[job_1mh1_gnp_script_20251212] Merged 1 metal ions into PQR.


/usr/local/miniforge3/envs/htvs/lib/python3.12/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')


[job_1mh1_gnp_script_20251212] Converted PQR -> PDB using pybel: /home/daneel/gitrepos/gromacs_sims/htvs/1mh1_gnp/job_1mh1_gnp_script_20251212/1mh1_protein_h.pdb
[job_1mh1_gnp_script_20251212] Executing: prepare_receptor4 -r 1mh1_protein_h.pdb -C -p MG -U nphs_lps_waters_nonstdres_deleteAltB -o 1mh1_protein_h.pdbqt


 charge : 13.354
 radius : 0.2943
 charge : 12.55
 radius : -0.01
 charge : 13.483
 radius : 0.6163
 charge : 14.7
 radius : -0.5722
 charge : 13.675
 radius : 0.1642
 charge : 12.258
 radius : 0.0895
 charge : 11.791
 radius : 0.0895
 charge : 12.783
 radius : 0.1642
 charge : 14.133
 radius : 0.1642
 charge : 14.441
 radius : -0.2548
 charge : 14.227
 radius : -0.0266
 charge : 12.903
 radius : 0.5896
 charge : 12.668
 radius : -0.5748
 charge : 15.437
 radius : -0.007
 charge : 16.473
 radius : 0.0189
 charge : 15.68
 radius : 0.0192
 charge : 14.174
 radius : 0.0641
 charge : 15.24
 radius : 0.0253
 charge : 15.756
 radius : 0.0253
 charge : 17.18
 radius : 0.0213
 charge : 16.875
 radius : 0.0213
 charge : 15.479
 radius : 0.0391
 charge : 16.126
 radius : 0.0391
 charge : 12.939
 radius : -0.4157
 charge : 13.759
 radius : -0.0249
 charge : 13.54
 radius : 0.5973
 charge : 12.551
 radius : -0.5679
 charge : 13.368
 radius : 0.2117
 charge : 13.038
 radius : -0.6546
 charge : 12.3

[job_1mh1_gnp_script_20251212] Prepare_Receptor4 completed successfully.
[job_1mh1_gnp_script_20251212] Executing: prepare_ligand4 -l /home/daneel/gitrepos/gromacs_sims/htvs/1mh1_gnp/gnp.pdb -o gnp.pdbqt
[job_1mh1_gnp_script_20251212] Prepare_Ligand4 completed successfully.
[job_1mh1_gnp_script_20251212] Executing: autogrid4 -p grid.gpf -l grid.glg
[job_1mh1_gnp_script_20251212] AutoGrid4 completed successfully.
[job_1mh1_gnp_script_20251212] --- Running AutoLigand (Automatic Mode) ---
[job_1mh1_gnp_script_20251212] AutoLigand command executed. Parsing generated FILL files...
[job_1mh1_gnp_script_20251212] AutoLigand identified 7 binding site(s).
[job_1mh1_gnp_script_20251212] Executing: vina --receptor 1mh1_protein_h.pdbqt --ligand gnp.pdbqt --center_x 5.49811111111112 --center_y 44.92277777777769 --center_z 16.4458888888889 --size_x 30 --size_y 30 --size_z 30 --cpu 12 --out gnp_pose_site1.pdbqt > vina_site1.log 2>&1
[job_1mh1_gnp_script_20251212] AutoDock Vina (Site 1) completed succ

In [2]:
from htvs_amdock import analyze_htvs_results
analyze_htvs_results(metalloprotein_job)


--- Summary for 1mh1_protein + gnp ---

=== DOCKING LOG SUMMARY ===

Log file: /home/daneel/gitrepos/gromacs_sims/htvs/1mh1_gnp/job_1mh1_gnp_script_20251212/1mh1_protein_gnp_pipeline.log

Detected tools / headers:
  - propka_header: INFO:PROPKA information:
  - pdb2pqr: 3.6.1
  - vina: present

PDB2PQR / structure info:
  Residues: 184, Atoms: 1379
  Unknown residue/component definitions: MG.
  Missing atoms detected: 111 (see warnings)

PROPKA / pKa application:
  pH applied: 7.4
  Calculated pI (folded): 6.79
  ======== PDB2PQR/PROPKA STDOUT ========
  pKa entries parsed: 60
  Lowest pKa residues (up to 5):
         ASP 47A  pKa=2.22
         ASP 65A  pKa=2.39
        ASP 122A  pKa=2.50
        GLU 148A  pKa=2.94
         C- 181A  pKa=3.33
  Highest pKa residues (up to 5):
         ARG 94A  pKa=15.13
        ARG 174A  pKa=15.11
         TYR 72A  pKa=14.76
        TYR 154A  pKa=14.43
        ARG 120A  pKa=13.88

AutoLigand (Automatic) site search:
  Rank 1: Volume=260.0, EPV=-0.2362


### Report (20251213)

So this particular metalloprotein complex does dock after a lot of painful vibe coding by trial and error, but the docking is inferior to `AMDock`.
The most likely culprit is the `-C` option executed while calling `prepare_receptor4`, which preserves all input charges ie do not add new charges. This overrides the default option of adding gasteiger charges and messes up the charges of the atoms.

However, eliminating this option results in `prepare_receptor4` crashing, suggesting that the bloody metal entries are still being malformatted somehow.

Will have to vibe code this further.


Meanwhile, the new code still works on non-metal proteins, as seen below.

In [3]:
#/usr/bin/env python
import importmonkey
import os
home = os.environ["HOME"]

importmonkey.add_path(os.path.join(home, "gitrepos/gromacs_sims/scripts"))
from htvs_amdock import run_htvs

PROTEIN_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1fkb_rap/1FKB_expt.pdb")
LIGAND_PDB = os.path.join(home, "gitrepos/gromacs_sims/htvs/1fkb_rap/rap.pdb")

new_1fkb_rap = [{
            "protein_name": "1fkb_protein",
            "ligand_name": "rap",
            "protein_pdb": PROTEIN_PDB,
            "ligand_pdb": LIGAND_PDB,
            "working_dir": os.path.join(home, "gitrepos/gromacs_sims/htvs/1fkb_rap/job_1fkb_rap_script_20251213"),
        }]


# The docking 
run_htvs(new_1fkb_rap, verbose=False)

/usr/local/miniforge3/envs/htvs/lib/python3.12/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
 charge : 8.513
 radius : 0.2943
 charge : 7.474
 radius : -0.01
 charge : 7.981
 radius : 0.6163
 charge : 9.187
 radius : -0.5722
 charge : 9.34
 radius : 0.1642
 charge : 7.226
 radius : 0.0895
 charge : 6.664
 radius : 0.0895
 charge : 8.206
 radius : 0.1642
 charge : 8.746
 radius : 0.1642
 charge : 7.058
 radius : -0.4157
 charge : 7.337
 radius : -0.0875
 charge : 6.634
 radius : 0.5973
 charge : 5.471
 radius : -0.5679
 charge : 6.904
 radius : 0.2985
 charge : 5.431
 radius : -0.3192
 charge : 7.235
 radius : -0.3192
 charge : 6.085
 radius : 0.2719
 charge : 8.311
 radius : 0.0969
 charge : 7.441
 radius : -0.0297
 charge : 4.951
 radius : 0.0791
 charge : 5.355
 radius : 0.0791
 charge : 5.027
 radius

Summary: 1/1 pairs successfully processed.


In [4]:
from htvs_amdock import analyze_htvs_results
analyze_htvs_results(new_1fkb_rap)


--- Summary for 1fkb_protein + rap ---

=== DOCKING LOG SUMMARY ===

Log file: /home/daneel/gitrepos/gromacs_sims/htvs/1fkb_rap/job_1fkb_rap_script_20251213/1fkb_protein_rap_pipeline.log

Detected tools / headers:
  - propka_header: INFO:PROPKA information:
  - pdb2pqr: 3.6.1
  - vina: present

PDB2PQR / structure info:
  Residues: 107, Atoms: 1021

PROPKA / pKa application:
  pH applied: 7.4
  Calculated pI (folded): 8.67
  ======== PDB2PQR/PROPKA STDOUT ========
  pKa entries parsed: 36
  Lowest pKa residues (up to 5):
         ASP 11A  pKa=1.99
         ASP 37A  pKa=2.13
         ASP 79A  pKa=2.71
        ASP 100A  pKa=2.79
         C- 107A  pKa=2.89
  Highest pKa residues (up to 5):
         ARG 71A  pKa=14.39
         ARG 42A  pKa=14.31
         ARG 57A  pKa=13.56
         ARG 40A  pKa=12.83
         ARG 13A  pKa=12.46

AutoLigand (Automatic) site search:
  Rank 1: Volume=247.0, EPV=-0.1932
  Rank 2: Volume=295.0, EPV=-0.1640
  Rank 3: Volume=303.0, EPV=-0.1627
  Rank 4: Volume=2

Nonmetal proteins are docking fine.